In [1]:
# Cell 1: Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import math

In [2]:
# Load your data (replace with your actual data loading)
data = pd.read_excel("Daily.xlsx")
data.head()

,Date,Open,High,Low,Close,Tick Volume,Volume,Spread
0,2004.06.11,384.0,384.8,382.8,384.1,303,0,0
1,2004.06.14,384.3,385.8,381.8,382.8,1954,0,0
2,2004.06.15,382.8,388.8,381.1,388.6,1995,0,0
3,2004.06.16,387.1,389.8,382.6,383.8,2106,0,0
4,2004.06.17,383.6,389.3,383.0,387.6,1638,0,0


In [3]:
# Cell 3: Preprocess data
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
print(f"Data shape: {data.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")

Data shape: (5325, 7)
Date range: 2004-06-11 00:00:00 to 2025-03-05 00:00:00


In [4]:
# Cell 4: Scale the data
scaler = MinMaxScaler(feature_range=(-1, 1))  # Diffusion models work better with symmetric range
scaled_data = scaler.fit_transform(data[['Close']])
print(f"Scaled data range: {scaled_data.min():.4f} to {scaled_data.max():.4f}")

Scaled data range: -1.0000 to 1.0000


In [5]:
# Cell 5: Create sequences function
def create_sequences(data, seq_length):
    X = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
    return np.array(X)

In [6]:
# Create sequences
seq_length = 60
X = create_sequences(scaled_data, seq_length)
print(f"Sequences shape: {X.shape}")

Sequences shape: (5265, 60)


In [7]:
# Cell 6: Split data
train_size = int(len(X) * 0.8)
X_train = X[:train_size]
X_test = X[train_size:]
print(f"Training data: {X_train.shape}")
print(f"Test data: {X_test.shape}")

Training data: (4212, 60)
Test data: (1053, 60)


In [13]:
class DiffusionModel:
    def __init__(self, seq_length=60, hidden_dim=128, num_timesteps=1000):
        self.seq_length = seq_length
        self.hidden_dim = hidden_dim
        self.num_timesteps = num_timesteps
        self.beta_start = 1e-4
        self.beta_end = 0.02

        # Create noise schedule
        self.betas = np.linspace(self.beta_start, self.beta_end, num_timesteps)
        self.alphas = 1.0 - self.betas
        self.alpha_bars = np.cumprod(self.alphas)

        # Build the model
        self.model = self._build_model()

    def _build_model(self):
        """Build the U-Net style denoising model"""
        # Input for noisy data
        noisy_input = Input(shape=(self.seq_length,), name='noisy_input')

        # Input for timestep
        timestep_input = Input(shape=(1,), name='timestep_input')

        # Timestep embedding
        timestep_embed = TimeStepEmbedding(embed_dim=self.hidden_dim)(timestep_input)

        # Encoder
        x = Dense(self.hidden_dim, activation='relu')(noisy_input)
        x = LayerNormalization()(x)

        # Add timestep information
        x = Concatenate()([x, timestep_embed])

        # Hidden layers with skip connections
        x1 = Dense(self.hidden_dim * 2, activation='relu')(x)
        x1 = Dropout(0.1)(x1)
        x1 = LayerNormalization()(x1)

        x2 = Dense(self.hidden_dim * 2, activation='relu')(x1)
        x2 = Dropout(0.1)(x2)
        x2 = LayerNormalization()(x2)

        # Skip connection
        x = Concatenate()([x1, x2])

        x3 = Dense(self.hidden_dim, activation='relu')(x)
        x3 = LayerNormalization()(x3)

        # Output layer - predict noise
        noise_pred = Dense(self.seq_length, activation='linear', name='noise_output')(x3)

        model = Model(inputs=[noisy_input, timestep_input], outputs=noise_pred)
        return model

    def _timestep_embedding(self, timestep, embed_dim=64):
        """Create sinusoidal timestep embeddings"""
        half_dim = embed_dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
        emb = tf.cast(timestep, tf.float32)[:, None] * emb[None, :]
        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=1)

        # Project to desired dimension
        emb = Dense(embed_dim, activation='relu')(emb)
        return emb

print("DiffusionModel class defined!")


DiffusionModel class defined!


In [17]:
# Cell 8: Define training methods
def add_noise(self, x, t):
    """Add noise to data at timestep t"""
    noise = tf.random.normal(shape=tf.shape(x))
    alpha_bar_t = tf.gather(self.alpha_bars, t)
    alpha_bar_t = tf.reshape(alpha_bar_t, [-1, 1])

    noisy_x = tf.sqrt(alpha_bar_t) * x + tf.sqrt(1 - alpha_bar_t) * noise
    return noisy_x, noise

def train_step(self, x_batch):
    """Single training step"""
    batch_size = tf.shape(x_batch)[0]

    # Sample random timesteps
    t = tf.random.uniform([batch_size], 0, self.num_timesteps, dtype=tf.int32)

    # Add noise
    noisy_x, noise = self.add_noise(x_batch, t)

    with tf.GradientTape() as tape:
        # Predict noise
        noise_pred = self.model([noisy_x, tf.cast(t, tf.float32)[:, None]], training=True)

        # Compute loss
        loss = tf.reduce_mean(tf.square(noise - noise_pred))

    # Update weights
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

    return loss

# Add methods to the class
DiffusionModel.add_noise = add_noise
DiffusionModel.train_step = train_step


In [18]:
# Cell 9: Define sampling methods
def sample(self, shape, num_inference_steps=50):
    """Generate samples using DDPM sampling"""
    # Start with pure noise
    x = tf.random.normal(shape)

    # Create sampling schedule
    timesteps = np.linspace(self.num_timesteps - 1, 0, num_inference_steps, dtype=int)

    for i, t in enumerate(timesteps):
        t_batch = tf.fill([shape[0]], t)

        # Predict noise
        noise_pred = self.model([x, tf.cast(t_batch, tf.float32)[:, None]], training=False)

        # Compute denoising step
        alpha_t = self.alphas[t]
        alpha_bar_t = self.alpha_bars[t]
        beta_t = self.betas[t]

        # Compute coefficients
        coeff1 = 1 / tf.sqrt(alpha_t)
        coeff2 = beta_t / tf.sqrt(1 - alpha_bar_t)

        # Update x
        x = coeff1 * (x - coeff2 * noise_pred)

        # Add noise (except for the last step)
        if i < len(timesteps) - 1:
            noise = tf.random.normal(tf.shape(x))
            sigma_t = tf.sqrt(beta_t)
            x = x + sigma_t * noise

    return x

def compile_model(self, learning_rate=1e-4):
    """Compile the model"""
    self.optimizer = Adam(learning_rate=learning_rate)

# Add methods to the class
DiffusionModel.sample = sample
DiffusionModel.compile_model = compile_model

In [14]:
from tensorflow.keras.layers import Layer

class TimeStepEmbedding(Layer):
    def __init__(self, embed_dim=64, **kwargs):
        super(TimeStepEmbedding, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense = Dense(embed_dim, activation='relu')

    def call(self, timestep):
        half_dim = self.embed_dim // 2
        emb = tf.math.log(10000.0) / (half_dim - 1)
        emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
        emb = tf.cast(timestep, tf.float32) * tf.expand_dims(emb, 0)
        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
        return self.dense(emb)


In [19]:
# Cell 10: Create and compile the model
diffusion_model = DiffusionModel(seq_length=seq_length, hidden_dim=128, num_timesteps=1000)
diffusion_model.compile_model(learning_rate=1e-4)

# Display model summary
diffusion_model.model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ noisy_input         │ (None, 60)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │      7,808 │ noisy_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ timestep_input      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128)       │        256 │ dense_6[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_step_embeddin… │ (None, 128)       │     16,512 │ timestep_input[0… │
│ (TimeStepEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 256)       │          0 │ layer_normalizat… │
│ (Concatenate)       │                   │            │ time_step_embedd… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 256)       │     65,792 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dropout_2[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256)       │     65,792 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 256)       │          0 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dropout_3[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 512)       │          0 │ layer_normalizat… │
│ (Concatenate)       │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 128)       │     65,664 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128)       │        256 │ dense_9[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ noise_output        │ (None, 60)        │      7,740 │ layer_normalizat… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 230,844 (901.73 KB)

 Trainable params: 230,844 (901.73 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Cell 11: Define training loop
def fit_diffusion_model(model, X_train, epochs=50, batch_size=32, verbose=1):
    """Train the diffusion model"""
    dataset = tf.data.Dataset.from_tensor_slices(X_train)
    dataset = dataset.batch(batch_size).shuffle(1000)

    losses = []

    for epoch in range(epochs):
        epoch_losses = []

        for batch in dataset:
            loss = model.train_step(batch)
            epoch_losses.append(loss.numpy())

        avg_loss = np.mean(epoch_losses)
        losses.append(avg_loss)

        if verbose and (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.6f}')

    return losses

In [21]:
# Cell 12: Train the model
print("Training diffusion model...")
losses = fit_diffusion_model(diffusion_model, X_train, epochs=50, batch_size=32, verbose=1)

Training diffusion model...


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Mul] name: 

In [ ]:
# Cell 13: Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(losses)
plt.title('Diffusion Model Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Cell 14: Generate predictions
print("Generating predictions...")
num_test_samples = min(100, len(X_test))
predictions = []

# Generate predictions for test sequences
for i in range(num_test_samples):
    # Generate sample
    generated = diffusion_model.sample(shape=(1, seq_length), num_inference_steps=50)
    predictions.append(generated[0, -1].numpy())

predictions = np.array(predictions).reshape(-1, 1)
actual = X_test[:num_test_samples, -1].reshape(-1, 1)

print(f"Generated {len(predictions)} predictions")

In [ ]:
# Cell 15: Inverse transform and calculate metrics
# Inverse transform predictions
predictions_original = scaler.inverse_transform(predictions)
actual_original = scaler.inverse_transform(actual)

# Calculate metrics
r2 = r2_score(actual_original, predictions_original)
rmse = np.sqrt(mean_squared_error(actual_original, predictions_original))

print(f"\nDiffusion Model Results:")
print(f"R² Score: {r2:.6f}")
print(f"RMSE: {rmse:.2f}")

In [ ]:
# Cell 16: Define RSMPE function (same as original)
def rsmpescore(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    symmetric_errors = np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2)
    smpe = np.mean(symmetric_errors)
    rsmpe = np.sqrt(smpe)
    return rsmpe

# Calculate RSMPE
rsmpe = rsmpescore(actual_original, predictions_original)
print(f"RSMPE: {rsmpe*100:.2f}%")

In [ ]:
# Cell 17: Visualize predictions vs actual
plt.figure(figsize=(15, 8))

# Plot 1: Predictions vs Actual (first 50 samples)
plt.subplot(2, 2, 1)
plt.plot(actual_original[:50], label='Actual', alpha=0.8, linewidth=2)
plt.plot(predictions_original[:50], label='Predicted', alpha=0.8, linewidth=2)
plt.title('Diffusion Model: Predictions vs Actual (First 50 samples)')
plt.xlabel('Sample Index')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Scatter plot
plt.subplot(2, 2, 2)
plt.scatter(actual_original, predictions_original, alpha=0.6)
plt.plot([actual_original.min(), actual_original.max()],
         [actual_original.min(), actual_original.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Scatter Plot')
plt.grid(True, alpha=0.3)

# Plot 3: Residuals
plt.subplot(2, 2, 3)
residuals = actual_original - predictions_original
plt.plot(residuals, alpha=0.7)
plt.title('Prediction Residuals')
plt.xlabel('Sample Index')
plt.ylabel('Residual (Actual - Predicted)')
plt.grid(True, alpha=0.3)

# Plot 4: Residuals histogram
plt.subplot(2, 2, 4)
plt.hist(residuals, bins=20, alpha=0.7, edgecolor='black')
plt.title('Residuals Distribution')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 18: Generate multiple samples to show uncertainty
print("Generating multiple samples to demonstrate uncertainty...")
num_samples = 10
uncertainty_predictions = []

# Generate multiple predictions for the same input
test_input = X_test[0:1]  # Take first test sample

for _ in range(num_samples):
    generated = diffusion_model.sample(shape=(1, seq_length), num_inference_steps=50)
    uncertainty_predictions.append(generated[0, -1].numpy())

uncertainty_predictions = np.array(uncertainty_predictions)
uncertainty_original = scaler.inverse_transform(uncertainty_predictions.reshape(-1, 1))

print(f"Multiple predictions for same input:")
print(f"Mean: {uncertainty_original.mean():.2f}")
print(f"Std:  {uncertainty_original.std():.2f}")
print(f"Min:  {uncertainty_original.min():.2f}")
print(f"Max:  {uncertainty_original.max():.2f}")

In [ ]:
# Cell 19: Compare with original LSTM results (if available)
print("\nComparison Summary:")
print("="*50)
print("Diffusion Model:")
print(f"  R² Score: {r2:.6f}")
print(f"  RMSE: {rmse:.2f}")
print(f"  RSMPE: {rsmpe*100:.2f}%")
print("\nOriginal LSTM (from your notebook):")
print("  R² Score: 0.978087")
print("  RMSE: 46.97")
print("  RSMPE: 13.67%")
print("="*50)